In [13]:
from datetime import timedelta
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [14]:
country_mapper = pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv')
country_mapper['name'] = country_mapper['name'].str.upper()
country_mapper = country_mapper[["name", "alpha-3"]]
country_dict = country_mapper.set_index('name')['alpha-3'].to_dict()

REST API Scraping for IEEE

In [15]:
memoization = dict()

In [40]:
all_records = []

def scrape_ieee(query, num_pages):
    url = "https://ieeexplore.ieee.org/rest/search"
    headers = {
        "accept": "application/json, text/plain, */*",
        "accept-language": "th-TH,th;q=0.9",
        "content-type": "application/json",
        "origin": "https://ieeexplore.ieee.org",
        "priority": "u=1, i",
        "referer": f"https://ieeexplore.ieee.org/search/searchresult.jsp?newsearch=true&queryText={query}",
        "sec-ch-ua": "\"Chromium\";v=\"124\", \"Google Chrome\";v=\"124\", \"Not-A.Brand\";v=\"99\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
        "x-security-request": "required"
    }

    for page in range(1, num_pages+1):
        data = {
            "newsearch": True,
            "queryText": query,
            "highlight": True,
            "returnFacets": ["ALL"],
            "returnType": "SEARCH",
            "matchPubs": True,
            "pageNumber": page
        }

        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an exception for unsuccessful requests
        
        data = response.json()
        records = data.get('records', [])
        all_records.extend(records)

    # Create DataFrame from all records
    df = pd.json_normalize(all_records)
    return df

# EDIT the search query here
queries = ["Medical", "Engineering", "Biochemical"]

# EDIT pagination here
num_pages = 40

selected_columns = ['authors', 'publicationNumber', 'publicationDate', 'articleNumber',
                    'articleTitle', 'downloadCount',  'abstract', 'articleContentType']
for query in queries :
    df = scrape_ieee("query", num_pages)

selected_df = df[selected_columns]

def get_country(data, authorId):
    global memoization
    if not len(data):
        memoization[authorId] = None
        return None
    currentAffilations = data[0].get('currentAffiliations', [])
    if not len(currentAffilations):
        memoization[authorId] = None
        return None
    currentAffilations = currentAffilations[0]
    country = currentAffilations.split(', ')
    if not len(country):
        memoization[authorId] = None
        return None
    country = country[-1]
    for k in country_dict.keys():
        if country.upper() in k or k in country.upper():
            memoization[authorId] = country_dict[k]
            return country_dict[k]
    if country.upper() in country_dict.values():
        memoization[authorId] = country.upper()
        return country.upper()
    memoization[authorId] = None
    return None

def scrape_each_author(author):
    if('id' not in author) :
        return None
    authorId = author['id']
    global memoization
    if authorId in memoization.keys():
        return memoization[authorId]

    url = f"https://ieeexplore.ieee.org/rest/author/{authorId}"
    headers = {
        "accept": "application/json, text/plain, */*",
        "accept-language": "th-TH,th;q=0.9",
        "content-type": "application/json",
        "origin": "https://ieeexplore.ieee.org",
        "priority": "u=1, i",
        "referer": "https://ieeexplore.ieee.org/search/searchresult.jsp?newsearch=true&queryText=Engineering",
        "sec-ch-ua": "\"Chromium\";v=\"124\", \"Google Chrome\";v=\"124\", \"Not-A.Brand\";v=\"99\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        memoization[authorId] = None
        return None
    response.raise_for_status()  # Raise an exception for unsuccessful requests

    data = response.json()
    country = get_country(data, authorId)
    return country
    

selected_df['authors']

# selected_df['authorsName'] = selected_df['authors'].apply(lambda x: [author['preferredName'] for author in x] if type(x) == list else [])
selected_df['authorsAffilationCountry'] = selected_df['authors'].apply(lambda x: [scrape_each_author(author) for author in x])
# # selected_df = selected_df.drop(columns=['authors'], axis=1)
# print(selected_df.shape)
selected_df

/var/folders/nr/1q8cfq1915dfv1ctg_jszssh0000gn/T/ipykernel_35979/3616511408.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['authorsAffilationCountry'] = selected_df['authors'].apply(lambda x: [scrape_each_author(author) for author in x])


,authors,publicationNumber,publicationDate,articleNumber,articleTitle,downloadCount,abstract,articleContentType,authorsAffilationCountry
0,"[{'preferredName': 'Md. Shake Farid Uddin', 'n...",10440723,13-15 Dec. 2023,10441199,Analysis of SARGable Queries for Query Optimiz...,14,Being lived in the era of the 4th Industrial R...,Conferences,"[BGD, BGD, BGD]"
1,"[{'preferredName': 'Khin Myat Kyu', 'normalize...",9261768,4-5 Nov. 2020,9261805,Enhancement of Query Execution Time in SPARQL ...,143,"During couples of decades, the popularity of S...",Conferences,"[MMR, MMR]"
2,"[{'preferredName': 'Madhuri A. Potey', 'normal...",6495610,22-23 Feb. 2013,6514421,A survey of query log processing techniques an...,503,[::Query::] log is the pouch of valuable infor...,Conferences,"[IOT, IOT, IOT]"
3,"[{'preferredName': 'Xiaoyi Zhou', 'normalizedN...",8272882,9-10 Dec. 2017,8283258,Multi-query Optimization via Common Sub Query ...,144,"As the main [::query::] language on RDF data, ...",Conferences,"[CHN, CHN, CHN]"
4,"[{'preferredName': 'Jessie Ooi', 'normalizedNa...",7322100,19-21 Aug. 2015,7333094,"A survey of query expansion, query suggestion ...",777,The ineffectiveness of information retrieval s...,Conferences,"[MYS, MYS, MMR, MYS]"
...,...,...,...,...,...,...,...,...,...
2995,"[{'preferredName': 'Jiahui Yang', 'normalizedN...",9780876,20-22 Dec. 2021,9781051,Survey on Query Optimization of GPU database,231,"In the era of artificial intelligence, CPU-dri...",Conferences,"[CHN, CHN, CHN]"
2996,"[{'preferredName': 'Shauli Sarmin Sumi', 'norm...",8584037,1-3 Nov. 2018,8614984,Strategies for Continuous Region Query Process...,42,"Nowadays, many mobile applications provide loc...",Conferences,"[CAN, CAN]"
2997,"[{'preferredName': 'Magnus Bakken', 'normalize...",9613137,7-10 Sept. 2021,9613487,Quarry: An open source tool for OPC UA SPARQL ...,108,OPC UA is an open set of specifications for co...,Conferences,[NOR]
2998,"[{'preferredName': 'Nicolle Chaves Cysneiros',...",7491933,16-20 May 2016,7495627,Including hierarchical navigation in a Graph D...,220,"Distributed and replicated systems, such as Bi...",Conferences,"[None, None]"


In [42]:
finished_df = selected_df
finished_df['extracted_class'] = ''

finished_df.loc[:999, 'extracted_class'] = 'MEDI'
finished_df.loc[1000:1999, 'extracted_class'] = 'ENGI'
finished_df.loc[2000:2999, 'extracted_class'] = 'BIOC'

finished_df

/var/folders/nr/1q8cfq1915dfv1ctg_jszssh0000gn/T/ipykernel_35979/3466194012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finished_df['extracted_class'] = ''


,authors,publicationNumber,publicationDate,articleNumber,articleTitle,downloadCount,abstract,articleContentType,authorsAffilationCountry,extracted_class
0,"[{'preferredName': 'Md. Shake Farid Uddin', 'n...",10440723,13-15 Dec. 2023,10441199,Analysis of SARGable Queries for Query Optimiz...,14,Being lived in the era of the 4th Industrial R...,Conferences,"[BGD, BGD, BGD]",MEDI
1,"[{'preferredName': 'Khin Myat Kyu', 'normalize...",9261768,4-5 Nov. 2020,9261805,Enhancement of Query Execution Time in SPARQL ...,143,"During couples of decades, the popularity of S...",Conferences,"[MMR, MMR]",MEDI
2,"[{'preferredName': 'Madhuri A. Potey', 'normal...",6495610,22-23 Feb. 2013,6514421,A survey of query log processing techniques an...,503,[::Query::] log is the pouch of valuable infor...,Conferences,"[IOT, IOT, IOT]",MEDI
3,"[{'preferredName': 'Xiaoyi Zhou', 'normalizedN...",8272882,9-10 Dec. 2017,8283258,Multi-query Optimization via Common Sub Query ...,144,"As the main [::query::] language on RDF data, ...",Conferences,"[CHN, CHN, CHN]",MEDI
4,"[{'preferredName': 'Jessie Ooi', 'normalizedNa...",7322100,19-21 Aug. 2015,7333094,"A survey of query expansion, query suggestion ...",777,The ineffectiveness of information retrieval s...,Conferences,"[MYS, MYS, MMR, MYS]",MEDI
...,...,...,...,...,...,...,...,...,...,...
2995,"[{'preferredName': 'Jiahui Yang', 'normalizedN...",9780876,20-22 Dec. 2021,9781051,Survey on Query Optimization of GPU database,231,"In the era of artificial intelligence, CPU-dri...",Conferences,"[CHN, CHN, CHN]",BIOC
2996,"[{'preferredName': 'Shauli Sarmin Sumi', 'norm...",8584037,1-3 Nov. 2018,8614984,Strategies for Continuous Region Query Process...,42,"Nowadays, many mobile applications provide loc...",Conferences,"[CAN, CAN]",BIOC
2997,"[{'preferredName': 'Magnus Bakken', 'normalize...",9613137,7-10 Sept. 2021,9613487,Quarry: An open source tool for OPC UA SPARQL ...,108,OPC UA is an open set of specifications for co...,Conferences,[NOR],BIOC
2998,"[{'preferredName': 'Nicolle Chaves Cysneiros',...",7491933,16-20 May 2016,7495627,Including hierarchical navigation in a Graph D...,220,"Distributed and replicated systems, such as Bi...",Conferences,"[None, None]",BIOC


In [43]:
finished_df.to_csv('./scraped_data.csv')